In [12]:
import numpy as np
import json
from StringIO import StringIO
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import geoplotlib as gp
import scipy as sp
import datetime
from geoplotlib.utils import BoundingBox
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.preprocessing import scale
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request

import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

%matplotlib inline

In [13]:
coop_stations = pd.read_csv('data/station meta repo/coop_meta.csv',skipinitialspace=True)
alrcrn_stations = pd.read_csv('data/station meta repo/alrcrn_meta.csv',skipinitialspace=True)
rcrn_stations = pd.read_csv('data/station meta repo/rcrn_meta.csv',skipinitialspace=True)
asos_stations = pd.read_csv('data/station meta repo/asos_meta.csv',skipinitialspace=True)
awos_stations = pd.read_csv('data/station meta repo/awos_meta.csv',skipinitialspace=True)
nexrad_stations = pd.read_csv('data/station meta repo/nexrad_meta.csv',skipinitialspace=True)
lcd_stations = pd.read_csv('data/station meta repo/lcd_meta.csv',skipinitialspace=True)

#coop_stations, alrcrn_stations, rcrn_stations, asos_stations, awos_stations, nexrad_stations, lcd_stations

In [14]:
coop_copy=coop_stations[coop_stations.columns[:16]] 
coop_copy[coop_copy['FIPS_COUNTRY_NAME'].str.contains("UNITED STATES")]
coop_clean=coop_copy[['NCDCSTN_ID','COOP_ID','GHCND_ID','NAME_COOP_SHORT','FIPS_COUNTRY_NAME','STATE_PROV','COUNTY','LAT_DEC','LON_DEC','ELEV_GROUND','UTC_OFFSET']]
coop_clean.columns=['NCDCID','COOPID','GHCND','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']   ##rename columns
coop_clean['ELEV']=coop_clean['ELEV'].apply(lambda x: x/3.28).fillna(0).astype(int) ##convert feet to meters
print "COOP SAMPLE DATA" 
coop_clean.head()

COOP SAMPLE DATA


C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
0,20000075,10008,USC00010008,ABBEVILLE,UNITED STATES,AL,HENRY ...,31.57020,-85.24820,139,-6
1,20000284,10063,USC00010063,ADDISON,UNITED STATES,AL,WINSTON ...,34.25530,-87.18140,249,-6
2,10000485,10116,USW00053864,ALABASTER SHELBY CO AP,UNITED STATES,AL,SHELBY ...,33.17833,-86.78167,172,-6
3,20000116,10140,USC00010140,ALBERTA,UNITED STATES,AL,WILCOX ...,32.23220,-87.41040,53,-6
4,20000189,10160,USC00010160,ALEXANDER CITY,UNITED STATES,AL,TALLAPOOSA ...,32.94520,-85.94800,195,-6


In [15]:
awos_copy=awos_stations[awos_stations['COUNTRY'].str.contains("UNITED STATES") & awos_stations['COOPID'].notnull()]
awos_copy[['WBAN','COOPID']]=awos_copy[['WBAN','COOPID']].astype(int)
awos_clean=awos_copy[['NCDCID','COOPID','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']] ##get selected columns
awos_clean.insert(2, 'GHCND', "")
print "AWOS SAMPLE DATA" 
awos_clean.head()

AWOS SAMPLE DATA


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
48,20021772,502587,,DUTCH HARBOR,UNITED STATES,AK,ALEUTIANS WEST,53.89500,-166.54330,10,-9
57,10500011,503475,,GUSTAVUS,UNITED STATES,AK,SKAGWAY-HOONAH-ANGOON,58.41110,-135.70890,40,-9
84,20021817,507557,,PORT ALEXANDER,UNITED STATES,AK,SITKA BOROUGH,56.24720,-134.64470,12,-9
88,20022160,507783,,PUNTILLA,UNITED STATES,AK,MATANUSKA-SUSITNA BOROUGH,62.09110,-152.73490,1858,-9
279,20004861,94959,,LA GRANGE CAA AP,UNITED STATES,GA,TROUP,33.01667,-85.06667,682,-5


In [16]:
alrcrn_stations.head()
alrcrn_copy=alrcrn_stations[alrcrn_stations['COUNTRY'].str.contains("UNITED STATES") & alrcrn_stations['GHCND'].notnull()]
alrcrn_copy[['WBAN']]=alrcrn_copy[['WBAN']].astype(int)
alrcrn_clean=alrcrn_copy[['NCDCID','GHCND','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']] ##get selected columns
alrcrn_clean.insert(1, 'COOPID', 0)
print "ALRCRN SAMPLE DATA" 
alrcrn_clean.head()

ALRCRN SAMPLE DATA


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
0,30015397,0,USW00063899,BREWTON 3 NNE,UNITED STATES,AL,ESCAMBIA,31.1449,-87.0518,170,-6
1,30015398,0,USW00063891,CLANTON 2 NE,UNITED STATES,AL,CHILTON,32.8516,-86.6115,584,-6
2,30015400,0,USW00063868,COURTLAND 2 WSW,UNITED STATES,AL,LAWRENCE,34.6602,-87.3462,575,-6
3,30015401,0,USW00063867,CULLMAN 3 ENE,UNITED STATES,AL,CULLMAN,34.1954,-86.7963,800,-6
4,30015402,0,USW00063892,GAINESVILLE 2 NE,UNITED STATES,AL,SUMTER,32.8369,-88.1374,107,-6


In [17]:
rcrn_copy=rcrn_stations[rcrn_stations['COUNTRY'].str.contains("UNITED STATES") & rcrn_stations['GHCND'].notnull()]
rcrn_copy[['WBAN']]=rcrn_copy[['WBAN']].astype(int)
rcrn_clean=rcrn_copy[['NCDCID','GHCND','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']] ##get selected columns
rcrn_clean.insert(1, 'COOPID', 0)
print "RCRN SAMPLE DATA" 
rcrn_clean.head()

RCRN SAMPLE DATA


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
0,30015624,0,USW00053168,AJO 29 S,UNITED STATES,AZ,PIMA,31.9497,-112.8013,1661,-7
1,30015622,0,USW00053006,AMADO 23 W,UNITED STATES,AZ,PIMA,31.6922,-111.4481,3300,-7
2,30015416,0,USW00003085,BOWIE 23 SSE,UNITED STATES,AZ,COCHISE,32.0095,-109.3891,5133,-7
3,30015417,0,USW00053159,CAMERON 25 SSE,UNITED STATES,AZ,COCONINO,35.5037,-111.3445,4807,-7
4,30015418,0,USW00053160,CAMP VERDE 3 N,UNITED STATES,AZ,YAVAPAI,34.6136,-111.8431,3434,-7


In [18]:
asos_stations.head()
asos_copy=asos_stations[asos_stations['COUNTRY'].str.contains("UNITED STATES") & asos_stations['COOPID'].notnull()]
asos_copy[['WBAN','COOPID']]=asos_copy[['WBAN','COOPID']].notnull().astype(int)
asos_clean=asos_copy[['NCDCID','COOPID','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']] ##get selected columns
asos_clean.insert(2, 'GHCND', "")
print "ASOS SAMPLE DATA" 
asos_clean.head()

ASOS SAMPLE DATA


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
5,20022040,1,,ANCHORAGE TED STEVENS INTL AP,UNITED STATES,AK,ANCHORAGE BOROUGH,61.1690,-150.0278,120,-9
6,20021784,1,,ANNETTE WSO AP,UNITED STATES,AK,PRINCE OF WALES-OUTER KETCHIKA,55.0389,-131.5787,109,-9
7,20022476,1,,BARROW W POST-W ROGERS AP,UNITED STATES,AK,NORTH SLOPE BOROUGH,71.2834,-156.7815,31,-9
8,20022008,1,,BETHEL AP,UNITED STATES,AK,BETHEL,60.7850,-161.8293,102,-9
9,20022377,1,,BETTLES AP,UNITED STATES,AK,YUKON-KOYUKUK,66.9169,-151.5154,643,-9


In [19]:
nexrad_stations.head()
nexrad_copy=nexrad_stations[nexrad_stations['COUNTRY'].str.contains("UNITED STATES") & nexrad_stations['NCDCID'].notnull()]
nexrad_copy[['WBAN']]=nexrad_copy[['WBAN']].notnull().astype(int)
nexrad_clean=nexrad_copy[['NCDCID','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']] ##get selected columns
nexrad_clean.insert(1, 'COOPID', 0)
nexrad_clean.insert(2, 'GHCND', "")
print "NEXRAD SAMPLE DATA" 
nexrad_clean.head()

NEXRAD SAMPLE DATA


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
0,30001794,0,,ABERDEEN,UNITED STATES,SD,BROWN,45.45583,-98.41306,1302,-6
1,30001795,0,,ALBUQUERQUE,UNITED STATES,NM,BERNALILLO,35.14972,-106.82333,5870,-7
2,30001796,0,,NORFOLK RICH,UNITED STATES,VA,SUSSEX,36.98389,-77.00750,112,-5
3,30001797,0,,AMARILLO,UNITED STATES,TX,POTTER,35.23333,-101.70889,3587,-6
4,30001798,0,,MIAMI,UNITED STATES,FL,DADE,25.61056,-80.41306,14,-5


In [20]:
lcd_stations.head()
lcd_copy=lcd_stations[lcd_stations['FIPS_COUNTRY_CODE'].str.contains("US") & lcd_stations['COOP_ID'].notnull()]
lcd_copy[['WBAN']]=lcd_copy[['WBAN']].notnull().astype(int)
lcd_clean=lcd_copy[['COOP_ID','GHCND_ID','LOCATION','FIPS_COUNTRY_CODE','STATE_PROV','CITY','LAT_DEC','LON_DEC','ELEV_GROUND','UTC_OFFSET']] ##get selected columns
lcd_clean.columns=['COOPID','GHCND','NAME','COUNTRY','ST','COUNTY','LAT','LON','ELEV','UTC']   ##rename columns
lcd_clean['ELEV']=lcd_clean['ELEV'].apply(lambda x: x/3.28).fillna(0).astype(int) ##convert feet to meters
lcd_clean.insert(0, 'NCDCID', 0)
print "LCD SAMPLE DATA" 
lcd_clean.head()

LCD SAMPLE DATA


C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,NCDCID,COOPID,GHCND,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC
0,0,52211,USW00003017,DENVER INTERNATIONAL AIRPORT ...,US,CO,DENVER ...,39.8328,-104.6575,1650,-7
1,0,23010,USW00003103,FLAGSTAFF PULLIAM AIRPORT ...,US,AZ,FLAGSTAFF ...,35.1441,-111.6663,2135,-7
2,0,310300,USW00003812,ASHEVILLE REGIONAL AIRPORT ...,US,NC,ASHEVILLE ...,35.4319,-82.5375,645,-5
3,0,95443,USW00003813,MIDDLE GEORGIA REGIONAL AIRPORT ...,US,GA,MACON ...,32.6847,-83.6527,104,-5
4,0,156110,USW00003816,BARKLEY REGIONAL AIRPORT ...,US,KY,PADUCAH ...,37.0563,-88.7744,125,-6


### Concatanate all 7 trimed datasets into one integrated dataframe

In [21]:
all_stations=pd.concat([coop_clean, alrcrn_clean, rcrn_clean, asos_clean, awos_clean, nexrad_clean, lcd_clean], ignore_index=True)
all_stations['COUNTRY']
all_stations['COUNTRY'].replace('US','UNITED STATES',inplace=True,regex=True) ##trim US to UNITED STATES
print len(all_stations)
all_stations_clean=all_stations.drop_duplicates(subset=['LAT','LON'],keep='first')  ##discard duplicated by location
all_stations_clean=all_stations_clean[np.abs(all_stations_clean.LON-all_stations_clean.LON.mean())<=(3*all_stations_clean.LON.std())] ##discard OUTLIER by location  
print all_stations_clean.LON.max()

10192
-64.6494


In [ ]:
USA_COORDINATES = (37.0902, -95.7129)
opups, locations = [], []

width, height = 210,50

usa_station_map = folium.Map(location=USA_COORDINATES, zoom_start=3,
                   tiles='Stamen Terrain')

for index, row in all_stations_clean.iterrows():
    locations.append([row['LAT'], row['LON']])

t = folium.FeatureGroup(name='WeatherStation')
t.add_child(MarkerCluster(locations=locations))
usa_station_map.add_child(t)


usa_station_map.save('usa_station_map.html')
usa_station_map

In [35]:
token = "tfgFAatxdYQboXxrctwGYMyOjSFOvqAc"

import requests
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"
headers = {'token': token}
response = requests.get(url, headers = headers)
response = response.json()
print response

{u'results': [{u'elevation': 139, u'name': u'ABBEVILLE, AL US', u'maxdate': u'2014-01-01', u'datacoverage': 0.8813, u'longitude': -85.2482, u'latitude': 31.5702, u'elevationUnit': u'METERS', u'id': u'COOP:010008', u'mindate': u'1948-01-01'}, {u'elevation': 249.3, u'name': u'ADDISON, AL US', u'maxdate': u'2015-11-01', u'datacoverage': 0.5059, u'longitude': -87.1814, u'latitude': 34.2553, u'elevationUnit': u'METERS', u'id': u'COOP:010063', u'mindate': u'1938-01-01'}, {u'elevation': 302.1, u'name': u'ADDISON CENTRAL TOWER, AL US', u'maxdate': u'1962-03-01', u'datacoverage': 0.9658, u'longitude': -87.31667, u'latitude': 34.41667, u'elevationUnit': u'METERS', u'id': u'COOP:010071', u'mindate': u'1940-05-01'}, {u'elevation': 172.2, u'name': u'ALABASTER SHELBY CO AIRPORT, AL US', u'maxdate': u'2015-11-01', u'datacoverage': 0.8064, u'longitude': -86.78167, u'latitude': 33.17833, u'elevationUnit': u'METERS', u'id': u'COOP:010116', u'mindate': u'1995-04-01'}, {u'elevation': 183.8, u'name': u'BEL